Workshop Links:

Google Forms API: https://developers.google.com/workspace/forms/api/reference/rest/v1/forms

Google Cloud:
https://console.cloud.google.com/

In [ ]:
!pip install google-generativeai google-auth google-auth-oauthlib google-auth-httplib2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 93.5 MB/s eta 0:00:00


In [ ]:
# AI Quiz Generator with Google AI (Gemma + Google Forms API)

# Step 1: Import Required Libraries
import google.generativeai as genai
import google.auth
from googleapiclient.discovery import build
from google.colab import userdata
import json
import requests
import os
from google.colab import auth

In [ ]:
# Step 2: Authenticate and Setup APIs
PROJECT_ID = "630938461317"

auth.authenticate_user(project_id=PROJECT_ID)

# Set up API credentials
creds, _ = google.auth.default()
service = build('forms', 'v1', credentials=creds)

# Set up the model
GOOGLE_API_KEY = userdata.get('API_KEY1')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
# Step 3: Use Gemma to extract key topics and generate multiple-choice questions
def generate_quiz_questions(text):
    prompt = f"""Generate a 5 question multiple choice quiz with the full correct answers in the following format based on this text: {text}.
    Question 1: <question text>?
    a) <option 1>
    b) <option 2>
    c) <option 3>
    d) <option 4>
    Correct Answer: <correct answer>

    Question 2: <question text>?
    a) <option 1>
    b) <option 2>
    c) <option 3>
    d) <option 4>
    Correct Answer: <correct answer>

    """
    response = model.generate_content(prompt)
    return response.text  # Output generated questions and answers

In [ ]:
# Step 4: Create the quiz using requests to Google Forms API

def create_quiz(questions):
    # create the form with only the title
    form = {
        "info": {"title": "AI-Generated Science Quiz Test 2"},
    }
    result = service.forms().create(body=form).execute()
    form_id = result.get("formId")

    # update the form settings to Quiz mode
    update_settings = {
        "requests": {
            "updateSettings": {
                "settings": {
                    "quizSettings": {
                        "isQuiz": True
                    }
                },
                "updateMask": {
                    "paths": "quizSettings.isQuiz"
                }
            }
        }

    }
    service.forms().batchUpdate(formId=form_id, body=update_settings).execute()

    # Format the questions for the request
    requests_list = []
    index = 0
    for question_block in questions.split('\n\n'):
        if question_block.strip():
            lines = question_block.strip().split('\n')  # extract each line for that question
            question_text = lines[0].strip()  # get the question
            options = [line[line.find(')') + 2:].strip() for line in lines[1:len(lines)-1]]  # Extract options
            answer_line = lines[len(lines)-1].strip() # Extract line of correct answer
            correct_answer = answer_line[answer_line.find(')') + 2:].strip()  # Extract text of correct answer

            # Build the question item with correct answer for grading
            question_item = {
                "createItem": {
                    "item": {
                        "title": question_text,
                        "questionItem": {
                            "question": {
                                "required": True,
                                "choiceQuestion": {
                                    "type": "RADIO",
                                    "options": [{"value": option} for option in options],
                                },
                                # Add grading information
                                "grading": {
                                    "pointValue": 1,  # Assign points for correct answer
                                    "correctAnswers": {
                                        "answers": [{"value": correct_answer}]
                                    }
                                }
                            }
                        }
                    },
                    "location": {"index": index}  # Add at the end
                }
            }
            requests_list.append(question_item)
            index = index + 1

    # Execute the batch update
    body = {
        'requests': requests_list
    }
    # service.forms().batchUpdate(formId=form_id, body=body).execute()

    # form_url = f"https://docs.google.com/forms/d/{form_id}"
    # return form_url  # Return Google Form link

In [ ]:
# Step 5: Test with Sample Input
user_text = """Galilean and Cartesian theories about space, matter, and motion are at the foundation of the Scientific Revolution,
which is understood to have culminated with the publication of Newton's Principia Mathematica in 1687. Newton's theories about space and time helped him explain the
movement of objects. While his theory of space is considered the most influential in physics, it emerged from his predecessors' ideas about the same.
As one of the pioneers of modern science, Galileo revised the established Aristotelian and Ptolemaic ideas about a geocentric cosmos.
He backed the Copernican theory that the universe was heliocentric, with a stationary Sun at the center and the planets—including the Earth—revolving around the Sun.
If the Earth moved, the Aristotelian belief that its natural tendency was to remain at rest was in question.
Galileo wanted to prove instead that the Sun moved around its axis, that motion was as natural to an object as the state of rest.
In other words, for Galileo, celestial bodies, including the Earth, were naturally inclined to move in circles.
This view displaced another Aristotelian idea—that all objects gravitated towards their designated natural place-of-belonging."""
questions = generate_quiz_questions(user_text)
print(questions)

Question 1: Which scientific revolution figure is credited with the publication of *Principia Mathematica* in 1687, a work considered the culmination of the revolution?
a) Galileo Galilei
b) René Descartes
c) Ptolemy
d) Isaac Newton
Correct Answer: d) Isaac Newton

Question 2:  What was the central idea of the Copernican theory that Galileo supported?
a) A geocentric universe with the Earth at the center.
b) A heliocentric universe with the Sun at the center.
c) A universe with multiple centers of gravity.
d) A universe governed solely by Aristotelian principles.
Correct Answer: b) A heliocentric universe with the Sun at the center.

Question 3:  Which Aristotelian belief was challenged by Galileo's assertion that motion is as natural as rest?
a) The belief that the Sun revolves around the Earth.
b) The belief that the Earth is flat.
c) The belief that Earth's natural tendency is to remain at rest.
d) The belief that celestial bodies are made of a different substance than earthly objec

In [ ]:
quiz_link = create_quiz(questions)
print("Generated Quiz: ", quiz_link)  # Output Google Form link

Isaac Newton
A heliocentric universe with the Sun at the center.
The belief that Earth's natural tendency is to remain at rest.
To move in circles.
Newton's theory of space; Galileo and Descartes' ideas.
Generated Quiz:  https://docs.google.com/forms/d/1l7psotEcXgvG8hLcrcYdrjBfiD96nrxe6cR2zZK9Q58
